In [6]:
import xarray
import numpy as np
import pandas as pd

In [2]:
gbiomass_5min = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS.NC", engine="netcdf4")
glct = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")

gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'long lifetime timber                              ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'short lifetime timber                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'charcoal                                          ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'humus                                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'litter                                            ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'roots                                             ')

In [3]:
GLCT_1st = xarray.open_dataset('D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc', engine="netcdf4")

In [3]:
glct = glct.isel(time=slice(0, 11))
gbiomass_5min = gbiomass_5min.isel(time=slice(0, 13))

In [4]:
gbiomass_nbpsum = np.zeros((13, 2160, 4320), dtype='float32')

#LANGKAH 1c
for time_idx in range(13):
    gbiomass_nbpsum[time_idx] = gbiomass_5min['GBIOMASS'].isel(time=time_idx).sum(dim='NBP')

In [6]:
gbiomass_biomass_sumdim = xarray.Dataset({
        "gbiomass_biomass_sumdim":(["time", "latitude", "longitude"], gbiomass_nbpsum)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='2030-01-01', freq='5YS'),
        "latitude": gbiomass_5min.coords["latitude"].to_numpy(), 
        "longitude": gbiomass_5min.coords["longitude"].to_numpy(),
    })
gbiomass_biomass_sumdim

<xarray.Dataset>
Dimensions:                  (time: 13, latitude: 2160, longitude: 4320)
Coordinates:
  * time                     (time) datetime64[ns] 1970-01-01 ... 2030-01-01
  * latitude                 (latitude) float64 89.96 89.88 ... -89.87 -89.96
  * longitude                (longitude) float64 -180.0 -179.9 ... 179.9 180.0
Data variables:
    gbiomass_biomass_sumdim  (time, latitude, longitude) float32 0.0 0.0 ... 0.0

In [36]:
garea = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")

# Step 2a: Konversi unit GAREA dari km2 ke hektar
garea_1970 = garea.isel(time=slice(0, 13)) #slice 1975 to 2100
garea_ha = np.nan_to_num(np.multiply(garea_1970['GAREA'].to_numpy(), 100))

# Step 2b: Konversi unit selisih GBIOMASS dari Mg C/ km2 ke selected unit (ton C/ha)
#unit of gbiomass selisih (carbon density) adalah Mg C/ km2
#1 Mg = 1000 kg
#1000kg / 1 Mg = 1000
Mg_to_kg = 1e3
#1 Mg = 1 ton
Mg_to_tonne = 1
#1 km2 = 100 ha
km2_to_ha = 1e2

#Example conversion unit: 20.000 Mg C/km2 x (1 tonne/ 1 Mg)  / (100 ha/ 1km2) --> result in ton C/ha
gbiomass_tonCha = gbiomass_biomass_sumdim['gbiomass_biomass_sumdim'] * Mg_to_tonne / km2_to_ha

In [7]:
gbiomass_biomass_sumdim_manip = np.zeros((11, 2160, 4320), dtype='float32')

In [43]:
max_year = 11
for i in range(max_year): #1975
    if i <= max_year - 2:
        gbiomass_biomass_sumdim_manip[i+1] = np.where(
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_biof')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_natveg')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'biof_to_natveg')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_biof')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_natveg')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'natveg_to_agri')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'natveg_to_biof')|
                                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'natveg_to_grass'),
                                                  gbiomass_tonCha[i+2], 
                                                  gbiomass_tonCha[i+1])

In [46]:
gbiomass_biomass_sumdim_manip[0] = gbiomass_tonCha[0]

In [47]:
gbiomass_biomass_sumdim_manip_netcdf = xarray.Dataset({
        "gbiomass_biomass_sumdim_manip":(["time", "latitude", "longitude"], gbiomass_biomass_sumdim_manip)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gbiomass_biomass_sumdim.coords["latitude"].to_numpy(),
        "longitude": gbiomass_biomass_sumdim.coords["longitude"].to_numpy()
    })
gbiomass_biomass_sumdim_manip_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/gbiomass_biomass_sumdim_tonCha.nc", mode='w', format="NETCDF4")

In [1]:
import xarray
gbiomass_biomass_sumdim_manip_netcdf = xarray.open_dataset('D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/gbiomass_biomass_sumdim_tonCha.nc', engine="netcdf4")

In [3]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]
n = 4

gbiomass_biomass_sumdim_manip_netcdf['gbiomass_biomass_sumdim_manip'].sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

time
1970-01-01    214.590057
1975-01-01    208.643463
1980-01-01    204.567017
1985-01-01      0.999010
1990-01-01      0.999010
1995-01-01      1.244538
2000-01-01      1.274855
2005-01-01      1.296978
2010-01-01      1.359181
2015-01-01      1.413088
2020-01-01      1.423100
dtype: float32

In [7]:
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")
luh_static = xarray.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/gpw_v4_national_identifier_grid_rev11_5_min_finall.nc", engine="netcdf4")


ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((2160, 4320), dtype="<U40") #rubah ke <U64

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            
country = xarray.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": gbiomass_5min.coords["longitude"].to_numpy(), "latitude": gbiomass_5min.coords["latitude"].to_numpy()})

In [8]:
ds_new = glct.merge(gbiomass_biomass_sumdim_manip_netcdf, compat='no_conflicts', join='outer', fill_value=0)
ds_new = ds_new.assign_coords(country=country.country)
ds_new

<xarray.Dataset>
Dimensions:                        (longitude: 4320, latitude: 2160, time: 27)
Coordinates:
  * longitude                      (longitude) float64 -180.0 -179.9 ... 180.0
  * latitude                       (latitude) float64 89.96 89.88 ... -89.96
  * time                           (time) datetime64[ns] 1970-01-01 ... 2100-...
    country                        (latitude, longitude) <U40 'ocean' ... 'oc...
Data variables:
    GLCT_trans                     (time, latitude, longitude) object ...
    gbiomass_biomass_sumdim_manip  (time, latitude, longitude) float32 0.0 .....

In [9]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]
n = 4

ds_new.sel(latitude=y[n], longitude=x[n], method='nearest').to_dataframe()

,longitude,latitude,GLCT_trans,gbiomass_biomass_sumdim_manip,country
time,,,,,
1970-01-01,-96.625,18.541667,trof,214.590057,Mexico
1975-01-01,-96.625,18.541667,trof,208.643463,Mexico
1980-01-01,-96.625,18.541667,trof,204.567017,Mexico
1985-01-01,-96.625,18.541667,agri,0.999010,Mexico
1990-01-01,-96.625,18.541667,agri,0.999010,Mexico
1995-01-01,-96.625,18.541667,agri,1.244538,Mexico
2000-01-01,-96.625,18.541667,agri,1.274855,Mexico
2005-01-01,-96.625,18.541667,agri,1.296978,Mexico
2010-01-01,-96.625,18.541667,agri,1.359181,Mexico


In [25]:
result = np.where(ds_new['country'] == "Indonesia",
                  np.where(ds_new['GLCT_trans'].isel(time=0).to_numpy() == 'trof',
                           ds_new['gbiomass_biomass_sumdim_manip'].isel(time=0).to_numpy(), 0), 0)
result

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [50]:
mask = (ds_new['country'] == "Indonesia") & (ds_new['GLCT_trans'].isel(time=1).to_numpy() == 'trof') & (ds_new['gbiomass_biomass_sumdim_manip'].isel(time=1).to_numpy() > 0)
values = ds_new['gbiomass_biomass_sumdim_manip'].isel(time=0).to_numpy()[mask]
result = np.min(values) if values.size > 0 else 1
result

0.0

#percobaan indo trof 1970 = 287.03574
#percobaan indo trof 1975 = 287.6605
#percobaan indo trow 1970 = 174.0722

In [12]:
# sample code
label_map = [
    'agri',  'grass',  'carb',  'rfab',  'rftm',
    'biof',  'ice',  'tund',  'tuwd',  'bore',
    'ccfo',  'tmfo',  'tdfo',  'wmfo',  'stepp',
    'dsrt',  'scrb',  'svna',  'trow',  'trof'
]

nilai_list = list()
country_list = list()
label_list = list()
year_list = list()

# Mengambil data sekali saja di luar loop untuk efisiensi
data_country = ds_new['country'].to_numpy()
data_glct_all_time = ds_new['GLCT_trans'].to_numpy()
data_biomass_all_time = ds_new['gbiomass_biomass_sumdim_manip'].to_numpy()

# Mengambil daftar negara unik
# country_unique = np.unique(data_country)

# Iterasi melalui setiap kombinasi negara, label, dan tahun
# for negara in country_unique:
for label in label_map:
    for year, tahun in enumerate(range(1970, 1985, 5)):  # Asumsikan 11 tahun yang ada (time=0 s/d time=10)

        mask = np.where(data_country == "Indonesia",
                        np.where(data_glct_all_time[year] == label, data_biomass_all_time[year], 0),
                        0)
        nilai_max = mask.max()

        # Menyimpan hasil
        nilai_list.append(nilai_max)
        country_list.append("Indonesia")
        label_list.append(label)
        year_list.append(tahun)  # Misalkan interval tahun adalah 5 tahun

In [51]:
# sample code
label_map = [
    'agri',  'grass',  'carb',  'rfab',  'rftm',
    'biof',  'ice',  'tund',  'tuwd',  'bore',
    'ccfo',  'tmfo',  'tdfo',  'wmfo',  'stepp',
    'dsrt',  'scrb',  'svna',  'trow',  'trof'
]

nilai_list = list()
country_list = list()
label_list = list()
year_list = list()

# Mengambil data sekali saja di luar loop untuk efisiensi
data_country = ds_new['country'].to_numpy()
data_glct_all_time = ds_new['GLCT_trans'].to_numpy()
data_biomass_all_time = ds_new['gbiomass_biomass_sumdim_manip'].to_numpy()

# Mengambil daftar negara unik
# country_unique = np.unique(data_country)

# Iterasi melalui setiap kombinasi negara, label, dan tahun
# for negara in country_unique:
for label in label_map:
    for year, tahun in enumerate(range(1970, 1985, 5)):  # Asumsikan 11 tahun yang ada (time=0 s/d time=10)

        mask = (data_country == "Indonesia") & (data_glct_all_time[year] == label) & (data_biomass_all_time[year] > 0)
        values = data_biomass_all_time[year][mask]
        
        nilai_min = np.min(values) if values.size > 0 else 0

        # Menyimpan hasil
        nilai_list.append(nilai_min)
        country_list.append("Indonesia")
        label_list.append(label)
        year_list.append(tahun)  # Misalkan interval tahun adalah 5 tahun

In [ ]:
# full code max
label_map = [
    'agri',  'grass',  'carb',  'rfab',  'rftm',
    'biof',  'ice',  'tund',  'tuwd',  'bore',
    'ccfo',  'tmfo',  'tdfo',  'wmfo',  'stepp',
    'dsrt',  'scrb',  'svna',  'trow',  'trof'
]

nilai_list = list()
country_list = list()
label_list = list()
year_list = list()

# Mengambil data sekali saja di luar loop untuk efisiensi
data_country = ds_new['country'].to_numpy()
data_glct_all_time = ds_new['GLCT_trans'].to_numpy()
data_biomass_all_time = ds_new['gbiomass_biomass_sumdim_manip'].to_numpy()

# Mengambil daftar negara unik
country_unique = np.unique(data_country)

# Iterasi melalui setiap kombinasi negara, label, dan tahun
for negara in country_unique:
    for label in label_map:
        for year, tahun in enumerate(range(1970, 2025, 5)):  # Asumsikan 11 tahun yang ada (time=0 s/d time=10)

            mask = np.where(data_country == negara,
                            np.where(data_glct_all_time[year] == label, data_biomass_all_time[year], 0),
                            0)
            nilai_max = mask.max()

            # Menyimpan hasil
            nilai_list.append(nilai_max)
            country_list.append(negara)
            label_list.append(label)
            year_list.append(tahun)

In [ ]:
# full code min
label_map = [
    'agri',  'grass',  'carb',  'rfab',  'rftm',
    'biof',  'ice',  'tund',  'tuwd',  'bore',
    'ccfo',  'tmfo',  'tdfo',  'wmfo',  'stepp',
    'dsrt',  'scrb',  'svna',  'trow',  'trof'
]

nilai_list = list()
country_list = list()
label_list = list()
year_list = list()

# Mengambil data sekali saja di luar loop untuk efisiensi
data_country = ds_new['country'].to_numpy()
data_glct_all_time = ds_new['GLCT_trans'].to_numpy()
data_biomass_all_time = ds_new['gbiomass_biomass_sumdim_manip'].to_numpy()

# Mengambil daftar negara unik
country_unique = np.unique(data_country)

# Iterasi melalui setiap kombinasi negara, label, dan tahun
for negara in country_unique:
    for label in label_map:
        for year, tahun in enumerate(range(1970, 2025, 5)):  # Asumsikan 11 tahun yang ada (time=0 s/d time=10)

            mask = (data_country == negara) & (data_glct_all_time[year] == label) & (data_biomass_all_time[year] > 0)
            values = data_biomass_all_time[year][mask]
        
            nilai_min = np.min(values) if values.size > 0 else 0

            # Menyimpan hasil
            nilai_list.append(nilai_min)
            country_list.append(negara)
            label_list.append(label)
            year_list.append(tahun)

In [52]:
# Membuat DataFrame dari hasil
df = pd.DataFrame({
    'Year': year_list,
    'Country': country_list,
    'Label': label_list,
    'Value': nilai_list
})

# Menampilkan DataFrame
df

,Year,Country,Label,Value
0,1970,Indonesia,agri,0.717682
1,1975,Indonesia,agri,0.631847
2,1980,Indonesia,agri,0.746586
3,1970,Indonesia,grass,1.000000
4,1975,Indonesia,grass,1.000000
5,1980,Indonesia,grass,1.000000
6,1970,Indonesia,carb,1.000000
7,1975,Indonesia,carb,1.000000
8,1980,Indonesia,carb,1.000000
9,1970,Indonesia,rfab,1.000000


In [ ]:
# label_list = []
# year_list = []

# for label in label_map:
#     for year, tahun in enumerate(range(1970, 1980, 5)):  # Asumsikan 11 tahun yang ada (time=0 s/d time=10)
#         label_list.append(label)
#         year_list.append(tahun)  # Misalkan interval tahun adalah 5 tahun


# # Membuat DataFrame dari hasil
# df = pd.DataFrame({
#     'Year': year_list,
#     'Country': country_list,
#     'Label': label_list,
#     'Value': nilai_list
# })

# # Menampilkan DataFrame
# df

In [234]:
# # kode untuk semua grid tanpa looping negara hanya 220 an baris

# label_map = [
#     'agri',  'grass',  'carb',  'rfab',  'rftm',
#     'biof',  'ice',  'tund',  'tuwd',  'bore',
#     'ccfo',  'tmfo',  'tdfo',  'wmfo',  'stepp',
#     'dsrt',  'scrb',  'svna',  'trow',  'trof'
# ]

# nilai_list = list()
# country_list = list()

# for label in label_map:
#     for tahun in range(0, 11):
#         # Mengambil data untuk tahun tertentu
#         data_glct = ds_new['GLCT_trans'].isel(time=tahun).to_numpy()
#         data_biomass = ds_new['gbiomass_biomass_sumdim_manip'].isel(time=tahun).to_numpy()

#         # Menemukan nilai maksimum dan indeksnya
#         mask = np.where(data_glct == label, data_biomass, 0)
#         nilai_max = mask.max()
#         index_max = np.unravel_index(mask.argmax(), mask.shape)
        
#         # Menyimpan nilai maksimum
#         nilai_list.append(nilai_max)
        
#         # Mendapatkan koordinat dari indeks maksimum
#         lat_max = ds_new['latitude'][index_max[0]].item()
#         lon_max = ds_new['longitude'][index_max[1]].item()
        
#         # Menggunakan koordinat untuk mengambil nilai country
#         country = ds_new.sel(latitude=lat_max, longitude=lon_max, method='nearest')['country'].item()
#         country_list.append(country)

# # Hasil akhirnya adalah nilai_list dan country_list